# Compare H151 treated 5xFAD vs DMSO treated 5xFAD

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SeuratWrappers)
    library(patchwork)
    library(ggplot2)
    library(repr)
    library(gridExtra)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(tidyverse)
    library(ggrepel)
    library(scater)
})
options(future.globals.maxSize = 1e9)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [2]:
setwd("~/5XFAD_mouse/data/")

In [3]:
seurat_object <- readRDS(
    "mouseAD_H151_RNA_seurat_object.rds")

In [4]:
unique(seurat_object$genotype)

[1] "WT"    "5XFAD"

In [5]:
fad_object <- subset(seurat_object, subset = genotype == "5XFAD")

In [ ]:
for (cluster in unique(fad_object$final_celltype)) {
    curr_object <- fad_object[, fad_object$final_celltype == cluster]
    if (ncol(curr_object) < 10) {
        next
    }
    curr_counts <- LayerData(curr_object, layer = c("counts"))
    curr_meta <- curr_object@meta.data
    curr_sce <- SingleCellExperiment(assays = list(counts = curr_counts), colData = curr_meta)
    curr_sce$group <- factor(curr_sce$group, levels = c("5XFAD_DMSO", "5XFAD_H151"))
    groups <- curr_sce$mouse_id
    curr_aggr <- aggregateAcrossCells(curr_sce, groups, store.number = "group.size")
    
    curr_dge <- DGEList(counts = counts(curr_aggr), group = curr_aggr$group, remove.zeros = TRUE)
    keep <- filterByExpr(curr_dge, min.count = 5, min.prop = 0.5)
    curr_dge <- curr_dge[keep, , keep.lib.sizes=FALSE]
    curr_dge <- calcNormFactors(curr_dge, method = "TMM")
    
    curr_design <- model.matrix(~ 0 + curr_aggr$group)
    colnames(curr_design) <- levels(curr_aggr$group)
    curr_dge <- estimateDisp(curr_dge, curr_design, robust = TRUE)
    curr_fit <- glmQLFit(curr_dge, curr_design)
    curr_glf <- glmQLFTest(curr_fit, contrast = c(-1, 1))
    
    write.table(
        topTags(curr_glf, n = Inf)$table, 
        file.path("DGE_H151_5XFAD/", paste0(cluster, ".result.tsv")), 
        row.names = TRUE,
        col.names = TRUE,
        quote = FALSE,
        sep = "\t"
    )
}

In [6]:
unique(fad_object$final_cluster)

[1] DG             DAM            SUB_2          SUB_3          SUB_1         
 [6] IOL            SUB-ProS       OPC            CA1            CA2           
[11] HMG            CA3            Vip            MOL            NFOL          
[16] Pvalb          Chandelier     DAO            Meis2          Sncg          
[21] Lamp5          Sst            Astro          Endo           IFN           
[26] VLMC           SMC-Peri       PVM            Choroid-plexus
29 Levels: DG CA1 CA2 CA3 SUB_1 SUB_2 SUB_3 SUB-ProS Lamp5 Pvalb Sst ... Choroid-plexus

In [7]:
unique(fad_object$final_celltype)

[1] DG             Micro          SUB            IOL            OPC           
 [6] CA1            CA2-CA3        Vip            Oligo          Pvalb         
[11] Chandelier     Meis2          Sncg           Lamp5          Sst           
[16] Astro          Endo           VLMC           SMC-Peri       Macro         
[21] Choroid-plexus
21 Levels: DG CA1 CA2-CA3 SUB Lamp5 Pvalb Sst Vip Meis2 Sncg ... Choroid-plexus

In [8]:
for (cluster in unique(fad_object$final_cluster)) {
    if (!(cluster %in% c("SUB_1", "SUB_2", "SUB_3", "SUB-ProS", "CA2", "CA3"))) {
        next
    }
    curr_object <- fad_object[, fad_object$final_cluster == cluster]
    if (ncol(curr_object) < 10) {
        next
    }
    curr_counts <- LayerData(curr_object, layer = c("counts"))
    curr_meta <- curr_object@meta.data
    curr_sce <- SingleCellExperiment(assays = list(counts = curr_counts), colData = curr_meta)
    curr_sce$group <- factor(curr_sce$group, levels = c("5XFAD_DMSO", "5XFAD_H151"))
    groups <- curr_sce$mouse_id
    curr_aggr <- aggregateAcrossCells(curr_sce, groups, store.number = "group.size")
    
    curr_dge <- DGEList(counts = counts(curr_aggr), group = curr_aggr$group, remove.zeros = TRUE)
    keep <- filterByExpr(curr_dge, min.count = 5, min.prop = 0.5)
    curr_dge <- curr_dge[keep, , keep.lib.sizes=FALSE]
    curr_dge <- calcNormFactors(curr_dge, method = "TMM")
    
    curr_design <- model.matrix(~ 0 + curr_aggr$group)
    colnames(curr_design) <- levels(curr_aggr$group)
    curr_dge <- estimateDisp(curr_dge, curr_design, robust = TRUE)
    curr_fit <- glmQLFit(curr_dge, curr_design)
    curr_glf <- glmQLFTest(curr_fit, contrast = c(-1, 1))
    
    write.table(
        topTags(curr_glf, n = Inf)$table, 
        file.path("DGE_H151_5XFAD/", paste0(cluster, ".result.tsv")), 
        row.names = TRUE,
        col.names = TRUE,
        quote = FALSE,
        sep = "\t"
    )
}

Removing 7509 rows with all zero counts

Removing 12663 rows with all zero counts

Removing 10458 rows with all zero counts

Removing 11747 rows with all zero counts

Removing 12208 rows with all zero counts

Removing 8343 rows with all zero counts

